# Road Following - Train Model (道路に沿った走行 resnet18モデルの学習)

このノートブックでは、入力画像を読み込み、ターゲットに対応するx、y値のセットを出力するようにニューラルネットワークを学習します。

この学習では、PyTorchのResNet18モデルをJetBot用に改修してから、モデルを学習します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### データセットインスタンスの作成

[torch.utils.data.Dataset](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataset.py)クラスを継承して``__len__`` と ``__getitem__``関数を実装実装した``XYDataset``クラスを作成します。このクラスは、画像をロードするための役割と、画像ファイル名からx,y値の値をパースして取得します。[torch.utils.data.Dataset](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataset.py)を継承する事で、すべてのtorchデータユーティリティを使用する事ができます。

いくつかの変換（カラージッターなど）をデータセットにハードコーディングしました。カラージッターは画像の明るさ、コントラスト、彩度をランダムに変更します。  
ランダムに水平反転を有効にするオプション``random_hflips``を付けました。しかしこのサンプルではオプションが機能する実装はしておらず、常に水平反転を有効にしています。水平反転は「右車線にとどまる」必要が無い場合、つまり左右どちらの車線を通ってもいい場合に有効にすることでデータセットの特性が変わります。

* `__getitem__()`について
  * https://stackoverflow.com/questions/43627405/understanding-getitem-method
* Python3.6 Special method names
  * https://docs.python.org/3.6/reference/datamodel.html#special-method-names
* Python3.6 copy
  * https://docs.python.org/3.6/library/copy.html

In [ ]:
########################################
# Jpeg画像ファイル名からxの値を取得します。
# 左端は-1.0、右端は+1.0になるように値を変換してから返します。
########################################
def get_x(path):
    return (float(int(path[3:6])) - 50.0) / 50.0

########################################
# Jpeg画像ファイル名からyの値を取得します。
# 上端は-1.0、下端は+1.0になるように値を変換してから返します。
########################################
def get_y(path):
    return (float(int(path[7:10])) - 50.0) / 50.0

########################################
# jpeg画像データを学習可能なデータフォーマットに
# 変換して提供するtorchデータセットクラスを作成します。
########################################
class XYDataset(torch.utils.data.Dataset):
    
    ####################
    # __init__()はXYDatasetクラスをインスタンス化したときに実行されます。
    ####################
    def __init__(self, directory, random_hflips=False):
        ####################
        # 画像データがあるディレクトリパスをこの関数以外からもアクセスできるようにクラス変数self.directoryに保持します。
        # directory情報はこの__init__()関数内でしか使われていないため、クラス変数の利用は必須ではありません。
        ####################
        self.directory = directory
        
        ####################
        # 画像をランダムに左右反転させる処理を行うかどうかのフラグ情報を
        # この関数以外からもアクセスできるようにクラス変数self.random_hflipsに保持します。
        # しかし、変数名を検索すると、このフラグは利用されていないことがわかります。
        # __getitem__()関数でhflip()している部分があるので、その処理を行うかどうかのif文を追加することで、
        # このフラグを機能させることができます。
        ####################
        self.random_hflips = random_hflips
        
        ####################
        # すべてのJpeg画像のファイルパスを配列で取得します。
        ####################
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        
        ####################
        # 色合いをランダムに変更する機能を定義します。
        ####################
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    ####################
    # XYDatasetクラスにデータ件数（画像ファイル数）を返す機能を持たせます。
    # このクラスをインスタンス化したdataset変数はlen(dataset)とすることで
    # データ件数を返すことが可能になります。
    ####################
    def __len__(self):
        return len(self.image_paths)
    
    ####################
    # このクラスのインスタンス（このコードではdataset、train_dataset、test_dataset変数として定義）
    # に学習データ取得機能を持たせます。(テストデータも学習データと同じフォーマット。)
    # 学習データの取得は配列で取得が可能となり、
    # データへのアクセス毎にランダム処理や正規化が実行されます。
    ####################
    def __getitem__(self, idx):
        ####################
        # 取得する画像のファイルパスを特定します。
        ####################
        image_path = self.image_paths[idx]
        
        ####################
        # 画像ファイルを読み込みます。
        ####################
        image = PIL.Image.open(image_path)
        
        ####################
        # ファイル名として保存してあるx, y値を取得します。
        ####################
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        ####################
        # 50%の確率で水平反転を実行します。
        ####################
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)  # 画像を水平反転します。
            x = -x  # 画像を水平反転後、xの値も反転します。
        
        ####################
        # 画像をモデル学習の入力用データフォーマットに変換します。
        ####################
        image = self.color_jitter(image)  # 色合いをランダムに変更します。
        image = transforms.functional.resize(image, (224, 224))  # 画像サイズを224x224(HxW)にリサイズします。
        image = transforms.functional.to_tensor(image)  # Tensor型に変換します。(CHW、RGBスケール[0.0, 1.0]、グラフレイヤーの追加)
        
        ####################
        # 画像部分だけを取得し、RGBをBGRに変換します。
        # さらっと書いてますが、road followingの推論モデルは
        # OpenCVカメラ画像のBGRフォーマットを入力に使っています。
        # BGRフォーマットのデータを学習することで、推論時にRGBに変換する手間を減らすことができます。
        # (ImageNetで学習済みのResNet18はRGBフォーマットのデータで学習されています。
        # そのためroad followingの転移学習ではこの差分を調整してくれるように願いたくなるかもしれません。)
        # Tensor.numpy()はCHWのままnumpy.ndarray型に変換します。(image.shapeでCHWのままであることを確認できます。)
        # copy()はアドレス渡しか値渡しかの違いで、画像データの配列を複製して返す値渡しになります。
        ####################
        image = image.numpy()[::-1].copy()
        
        ####################
        # 再びTensor型に変換します。
        # torch.from_numpy()の引数imageはCHWであることが期待されています。
        ####################
        image = torch.from_numpy(image)
        
        ####################
        # ImageNetのパラメータで正規化します。
        ####################
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        
        return image, torch.tensor([x, y]).float()  # 学習用に処理した画像データと、x, y値を返します。

####################
# XYDatasetクラスをインスタンス化します。
# 水平反転フラグは機能していません。（このコードでは常にに有効となっています。）
####################
dataset = XYDataset('dataset_xy', random_hflips=False)

### トレーニングデータとテストデータに分ける
次に、データセットを*トレーニング用*と*テスト用*のデータセットに分割します。この例では、*トレーニング用*に90%, *テスト用*に10%で分けます。*テスト用*のデータセットは、学習中にモデルの精度を検証するために使用されます。

In [ ]:
test_percent = 0.1  # テスト用のデータ件数を全体のデータ件数のうちの10%と定義します。
num_test = int(test_percent * len(dataset))  # テスト用に使うデータ件数を求めます。
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])  # すべてのデータセットを、学習用とテスト用に分割します。

### バッチ処理で学習データとテストデータを読み込むためのデータローダーを作成

[torch.utils.data.DataLoader](https://github.com/pytorch/pytorch/blob/master/torch/utils/data/dataloader.py)クラスは、モデル学習中に次のデータ処理が完了出来るようにサブプロセスで並列処理にして実装します。  
データのシャッフル、バッチでのデータロードのために使用します。この例では、1回のバッチ処理で8枚の画像を使用します。これをバッチサイズと呼び、GPUのメモリ使用量と、モデルの精度に影響を与えます。

In [ ]:
########################################
# データセットを分割読込みするためのデータローダーを作成します。
########################################
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

### JetBot用にモデルを変更する

torchvisionで使用可能なImageNetデータセットで学習済みのResNet18モデルを使用します。

*転移学習*と呼ばれる手法で、すでに画像分類できる特徴を持つニューラルネットワーク層を、別の目的のために作られたモデルに適用することで、短時間で良好な結果を得られるモデルを作成することができます。

ResNet18の詳細: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

転移学習の詳細：https://www.youtube.com/watch?v=yofjFQddwHE

In [ ]:
########################################
# PyTorchで提供されているImageNetデータセットで学習済みのResNet18モデルを読込みます。
########################################
model = models.resnet18(pretrained=True)

ResNet18モデルはImageNetを学習するために作られているため、1000種類の画像分類が可能な出力を持っています。ResNet18モデル構造の全結合層(fully connected layer)を入れ替えて、JetBotで欲しい出力x,yの2種類を得られるモデル構造にします。  
ResNetのfc層は入力側に512ノードあり、出力側に1000ノードあります。出力側をx,yに2ノードの変更するため、以下のようにモデルのfc層を書き換えます。

また、デフォルトではモデルのweightはCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
########################################
# モデルの出力層をJetBotの衝突回避モデル用に置き換えます。
########################################
model.fc = torch.nn.Linear(512, 2)

デフォルトではモデルのweightの計算はCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
########################################
# GPU処理が可能な部分をGPUで処理するように設定します。
########################################
device = torch.device('cuda')
model = model.to(device)

### モデルの学習

70エポック学習し、各エポックでテストデータにおけるこれまでの最小損失と現在の損失を比較することにより、最小損失を更新した場合に保存します。  

> 1エポックは、私たちが用意したトレーニング用のデータ全部を1回学習することです。データローダーのbatch_sizeで指定した数の画像を一度に学習するミニバッチ処理を複数回実行することで1エポックが完了します。

**Collision Avoidance**の時は、「free(直進する)」or「blocked(旋回する)」それぞれに対する正解ラベルは`True`or`False`の 0 or 1 で定義できました。また、「free(直進する)」or「blocked(旋回する)」のうち、1つだけが`True`となるため**one hot value**として定義できました。**one hot value**を予測する場合、モデルの精度の定義はテストデータでの予測結果が、**「正解ラベルと一致している件数」÷「テストデータの総数」**となります。

**Road Following**では正解ラベルはx,yの2出力それぞれ[-1.0,1.0]のfloat型の範囲になります。このため、正解ラベルと一致しない場合が多くなり、精度の定義は難しくなります。この場合、最小損失を更新した場合にモデルを保存します。

* one hot valueの詳細
  * https://www.youtube.com/watch?v=v_4KWmkwmsU  
* 回帰モデルの評価指標の詳細
  * https://www.ritchieng.com/machine-learning-evaluate-linear-regression-model/#15.-Model-Evaluation-Metrics-for-Regression
* model.train()とmodel.eval()について
  * https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch

In [ ]:
NUM_EPOCHS = 70  # 学習するエポック数。
BEST_MODEL_PATH = 'best_steering_model_xy.pth'  # 学習結果を保存するファイル名。
best_loss = 1e9  # 検証用の初期損失は10の9乗(1000000000)を意味する1e9としておきます。

optimizer = optim.Adam(model.parameters())  # 確率的最適化のAdamを実装します。

########################################
# 学習を開始します。
########################################
for epoch in range(NUM_EPOCHS):
    
    model.train()  # モデルを学習モードに設定します。
    train_loss = 0.0  # 学習損失を0.0で初期化します。
    ####################
    # Adamに基づいた学習をバッチ毎に実行します。
    ####################
    for images, labels in iter(train_loader):  # batch_size分の学習データを読み込みます。
        images = images.to(device)  # 画像データをGPUメモリに転送します。
        labels = labels.to(device)  # ラベルデータをGPUメモリに転送します。
        optimizer.zero_grad()  # 最適化されたすべてのtorch.Tensorの勾配をゼロに設定します。
        outputs = model(images)  # batch_size分の予測を一度に実行します。
        loss = F.mse_loss(outputs, labels)  # batch_size分のモデルの予測結果と正解ラベルを照合して損失を計算します。
        train_loss += float(loss)  # 1epochの損失を求めるために、今回のバッチの損失を加算します。
        loss.backward()  # 各パラメータ毎の損失の勾配を計算します。
        optimizer.step()  # 計算された損失勾配で各パラメータの勾配を更新します。
    train_loss /= len(train_loader)  # 1epochの損失からデータ1件あたりの学習損失を求めます。
    
    model.eval()  # モデルをテストモードに設定します。
    test_loss = 0.0  # テスト損失を0.0で初期化します。
    ####################
    # テストをバッチ毎に実行します。
    # ここは学習に使わないデータで推論を実行することで、
    # モデルの精度を確認します。
    ####################
    for images, labels in iter(test_loader):  # batch_size分の学習データを読み込みます。
        images = images.to(device)  # 画像データをGPUメモリに転送します。
        labels = labels.to(device)  # ラベルデータをGPUメモリに転送します。
        outputs = model(images)  # batch_size分の予測を一度に実行します。
        loss = F.mse_loss(outputs, labels)  # 1epochの損失を求めるために、今回のバッチの損失を加算します。
        test_loss += float(loss)  # 1epochの損失を求めるために、今回のバッチの損失を加算します。
    test_loss /= len(test_loader)  # 1epochの損失からデータ1件あたりの学習損失を求めます。
    
    ####################
    # 今回のepoch学習のテスト結果がよければ保存します
    ####################
    print('%d, %f, %f' % (epoch, train_loss, test_loss))  # 学習損失とテスト損失をログに表示します。
    if test_loss < best_loss:  # 過去最小の損失を更新した場合は保存します。
        torch.save(model.state_dict(), BEST_MODEL_PATH)  # モデルを保存します。
        best_loss = test_loss  # 過去最小の損失を更新します。

学習が完了すると、``03_live_demo_build_trt_JP.ipynb``でTensorRTモデルに変換するために使う``best_steering_model_xy.pth``が生成されます。

## Next(次)
次は、``03_live_demo_build_trt_JP.ipynb``を実行し、学習したPytorchモデルをTensorRTモデルに変換します。
JetBot本体で学習する場合は、このノートブックを閉じてからJupyter左側にある「Running Terminals and Kernels」を選択して「02_train_model_JP.ipynb」の横にある「SHUT DOWN」をクリックしてJupyter Kernelをシャットダウンしてから[03_live_demo_build_trt_JP.ipynb](03_live_demo_build_trt_JP.ipynb)に進んでください。  